## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [146]:
import pandas as pd
import numpy as np

### Importar base de dados

In [147]:
!wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

--2025-08-25 21:31:50--  https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz [following]
--2025-08-25 21:31:50--  https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65019041 (62M) [application/octet-stream]
Saving to: ‘ml-20m-compact.tar.gz.6’

ml-20m-compact.tar. 100%[===================>]  62.01M   239MB/s    in 0.3s    

2025-08-25 21:31:51 (239 MB/s) - ‘ml-20m-compact.tar.gz.6’ saved [65019041/6501

In [148]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [149]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [150]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [151]:
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474


### Divisão da base em treino e teste

In [152]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [153]:
# Instala ou atualiza direto do GitHub
%pip install -U git+https://github.com/caserec/CaseRecommender.git


In [154]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN
from sklearn.metrics import mean_absolute_error, mean_squared_error

train.to_csv('train.csv', index=False, header=False, sep='\t')
test.to_csv('test.csv', index=False, header=False, sep='\t')

# Preparar o arquivo de atributos (tags)
item_attributes = movies_tags.groupby('movieId')['tag'].apply(lambda x: '|'.join(x)).reset_index()
item_attributes.to_csv('item_tags.csv', index=False, header=False, sep='\t')

ItemAttributeKNN(

    train_file='train.csv',
    test_file='test.csv',
    output_file='output_ex1.csv',
    metadata_file='item_tags.csv',
    similarity_metric='jaccard',
    k_neighbors=5,
    sep='\t'

).compute()

# Resultados
print("\nResultados da predição:")
results_ex1 = pd.read_csv('output_ex1.csv', sep='\t', names=['userId', 'movieId', 'prediction'])
print(results_ex1.head())

# MAE and RMSE
merged_results = pd.merge(results_ex1, test, on=['userId', 'movieId'])
mae_ex1 = mean_absolute_error(merged_results['rating'], merged_results['prediction'])
rmse_ex1 = mean_squared_error(merged_results['rating'], merged_results['prediction'])**0.5

print(f"\nMAE: {mae_ex1:.4f}")
print(f"RMSE: {rmse_ex1:.4f}")

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 14.265876 sec
>> metadata:: 231 items and 229 metadata (231 interactions) | sparsity:: 99.56%
prediction_time:: 0.611511 sec
Eval:: MAE: 0.931913 RMSE: 1.184732 

Resultados da predição:
   userId  movieId  prediction
0       0        0    4.863712
1       0        5    4.725958
2       1       13    4.631619
3       1       14    4.782424
4       1       16    4.188857

MAE: 0.9319
RMSE: 1.1847


### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [ ]:
!wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
!tar -xvzf ml-20m-features.tar.gz

In [156]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

No. movies: 433
Size of each word: 50


In [157]:
visualHistograms[0]

array([0.00948992, 0.00237248, 0.00355872, 0.01304864, 0.05338078,
       0.00711744, 0.05456702, 0.00355872, 0.0059312 , 0.00355872,
       0.0059312 , 0.00474496, 0.00711744, 0.04507711, 0.07591934,
       0.00355872, 0.1316726 , 0.05338078, 0.00830368, 0.        ,
       0.01304864, 0.00474496, 0.02253855, 0.00355872, 0.02728351,
       0.00237248, 0.02016607, 0.00830368, 0.0059312 , 0.00237248,
       0.00355872, 0.04033215, 0.01067616, 0.00237248, 0.00830368,
       0.05931198, 0.0059312 , 0.01067616, 0.02965599, 0.01423488,
       0.00711744, 0.02609727, 0.        , 0.00237248, 0.01423488,
       0.03084223, 0.02016607, 0.00118624, 0.08778173, 0.02253855])

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc.

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima.

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme.

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações.

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity).

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [158]:
from sklearn.metrics.pairwise import cosine_similarity

# Vetores de caracteristicas visuais
with open('./features/visual_histogram_50.arq', 'rb') as f:
    visual_histograms = pickle.load(f)

num_movies_in_df = len(df['movieId'].unique())
visual_histograms_subset = visual_histograms[:num_movies_in_df]

# Similaridade do cosseno
similarity_matrix = cosine_similarity(visual_histograms_subset)

# Matriz de similaridade
similarity_file_path = 'visual_similarity.csv'
with open(similarity_file_path, 'w') as f:
    for i in range(len(similarity_matrix)):
        for j in range(i + 1, len(similarity_matrix)):
            f.write(f"{i}\t{j}\t{similarity_matrix[i, j]}\n")

ItemAttributeKNN(

    train_file='train.csv',
    test_file='test.csv',
    output_file='output_ex2.csv',
    similarity_file=similarity_file_path,
    k_neighbors=5,
    sep='\t'

).compute()

# Resultados e calcular métricas
print("\nResultados da predição:")
results_ex2 = pd.read_csv('output_ex2.csv', sep='\t', names=['userId', 'movieId', 'prediction'])
print(results_ex2.head())

test_data_from_file = pd.read_csv('test.csv', sep='\t', names=['userId', 'movieId', 'rating', 'title'])
merged_results_ex2 = pd.merge(results_ex2, test_data_from_file, on=['userId', 'movieId'])

# Calcular MAE e RMSE
mae_ex2 = mean_absolute_error(merged_results_ex2['rating'], merged_results_ex2['prediction'])
rmse_ex2 = mean_squared_error(merged_results_ex2['rating'], merged_results_ex2['prediction'])**0.5

print(f"\nMAE: {mae_ex2:.4f}")
print(f"RMSE: {rmse_ex2:.4f}")

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 19.359826 sec
prediction_time:: 0.279432 sec
Eval:: MAE: 0.759443 RMSE: 0.988201 

Resultados da predição:
   userId  movieId  prediction
0       0        0    3.863712
1       0        5    4.725958
2       1       13    3.631619
3       1       14    3.782424
4       1       16    3.188857

MAE: 0.7594
RMSE: 0.9882


***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos.
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3.
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados.
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

In [169]:
import pandas as pd
import numpy as np

# obter os generos
movies_df = pd.read_csv('./dataset/movies_sample.csv')
movies_df['movieId'] = movies_df['movieId'].map(map_items) # Mapear novos IDs
movies_df = movies_df.dropna(subset=['movieId']) # Remover filmes que não estão no df
movies_df['movieId'] = movies_df['movieId'].astype(int)

movie_genres = movies_df.set_index('movieId')['genres'].to_dict()

def predict_relevance_bayes(user_id, item_id, train_df, movie_genres_dict):

    user_ratings = train_df[train_df['userId'] == user_id].copy()

    if user_ratings.empty:
        return "Usuário sem histórico", 0, 0

    user_ratings['relevant'] = user_ratings['rating'] >= 3
    relevant_count = user_ratings['relevant'].sum()
    not_relevant_count = len(user_ratings) - relevant_count

    total_ratings = len(user_ratings)
    p_relevant = (relevant_count + 1) / (total_ratings + 2)
    p_not_relevant = (not_relevant_count + 1) / (total_ratings + 2)

    try:
        target_genres = movie_genres_dict[item_id].split('|')

    except KeyError:
        return "Filme sem gênero", 0, 0

    log_p_item_given_relevant = 0
    log_p_item_given_not_relevant = 0

    all_genres_list = [genre for genres in user_ratings['movieId'].map(movie_genres_dict).dropna() for genre in genres.split('|')]
    vocab_size = len(set(all_genres_list))

    relevant_movies = user_ratings[user_ratings['relevant'] == True]['movieId']
    not_relevant_movies = user_ratings[user_ratings['relevant'] == False]['movieId']

    relevant_genres = [g for m in relevant_movies for g in movie_genres_dict.get(m, '').split('|')]
    not_relevant_genres = [g for m in not_relevant_movies for g in movie_genres_dict.get(m, '').split('|')]


    for genre in target_genres:

        genre_count_relevant = relevant_genres.count(genre)
        p_genre_relevant = (genre_count_relevant + 1) / (len(relevant_genres) + vocab_size)
        log_p_item_given_relevant += np.log(p_genre_relevant)

        genre_count_not_relevant = not_relevant_genres.count(genre)
        p_genre_not_relevant = (genre_count_not_relevant + 1) / (len(not_relevant_genres) + vocab_size)
        log_p_item_given_not_relevant += np.log(p_genre_not_relevant)

    # log das relevancias dos itens
    final_p_relevant = np.log(p_relevant) + log_p_item_given_relevant
    final_p_not_relevant = np.log(p_not_relevant) + log_p_item_given_not_relevant


    prediction = 'Relevante' if final_p_relevant > final_p_not_relevant else 'Não Relevante'
    value = final_p_relevant if prediction == 'Relevante' else final_p_not_relevant
    return prediction, final_p_relevant, final_p_not_relevant, value

# Testando com os 10 dados
for i in range(10):
    test_sample = test.iloc[i]
    user_to_test = test_sample['userId']
    item_to_test = test_sample['movieId']
    actual_rating = test_sample['rating']
    actual_relevance = 'Relevante' if actual_rating >= 3 else 'Não Relevante'

    pred, prob_rel, prob_not_rel, val_pred = predict_relevance_bayes(user_to_test, item_to_test, train, movie_genres)

    print(f"Teste para Usuário {user_to_test} e Filme {item_to_test} ({map_title[item_to_test]})")
    print(f"Previsão Bayesiana: {pred}")
    print(f"Relevância Real (Nota {actual_rating}): {actual_relevance}\n")

Teste para Usuário 1836 e Filme 60 (She's the Man (2006))
Previsão Bayesiana: Relevante
Relevância Real (Nota 2.5): Não Relevante

Teste para Usuário 8646 e Filme 33 (North by Northwest (1959))
Previsão Bayesiana: Não Relevante
Relevância Real (Nota 3.5): Relevante

Teste para Usuário 1464 e Filme 19 (Reservoir Dogs (1992))
Previsão Bayesiana: Relevante
Relevância Real (Nota 4.5): Relevante

Teste para Usuário 5315 e Filme 33 (North by Northwest (1959))
Previsão Bayesiana: Não Relevante
Relevância Real (Nota 5.0): Relevante

Teste para Usuário 6571 e Filme 18 (Life Is Beautiful (La Vita è bella) (1997))
Previsão Bayesiana: Não Relevante
Relevância Real (Nota 4.0): Relevante

Teste para Usuário 126 e Filme 10 (Dark City (1998))
Previsão Bayesiana: Relevante
Relevância Real (Nota 3.5): Relevante

Teste para Usuário 1924 e Filme 56 (Showgirls (1995))
Previsão Bayesiana: Relevante
Relevância Real (Nota 1.0): Não Relevante

Teste para Usuário 4376 e Filme 4 (Bowling for Columbine (2002))
Pr

***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier.

In [173]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MultiLabelBinarizer

df['relevant'] = (df['rating'] >= 3).astype(int)
df_genres = df.merge(movies_df[['movieId', 'genres']], on='movieId')

# MultiLabelBinarizer e criando features one-hot-encod para os gêneros
mlb = MultiLabelBinarizer()
genre_features = pd.DataFrame(mlb.fit_transform(df_genres['genres'].str.split('|')), columns=mlb.classes_, index=df_genres.index)
full_df = pd.concat([df_genres, genre_features], axis=1)

# Escolher um usuário
user_id_example = 10
rated_movies = full_df[full_df['userId'] == user_id_example]

# Filmes não avaliados
unrated_movies = full_df[~full_df['movieId'].isin(rated_movies['movieId'])].drop_duplicates(subset=['movieId'])

feature_cols = mlb.classes_
X_train = rated_movies[feature_cols]
X_test = unrated_movies[feature_cols]

# MLPClassifier
print("* MLPClassifier")

y_train_class = rated_movies['relevant']
mlp_classifier = MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42, activation='relu', solver='adam')
mlp_classifier.fit(X_train, y_train_class)

relevance_probs = mlp_classifier.predict_proba(X_test)[:, 1]
unrated_movies['relevance_prob'] = relevance_probs
top_k_classifier = unrated_movies.sort_values('relevance_prob', ascending=False).head(10)

print(f"Top 10 filmes recomendados (por probabilidade de relevância) para o usuário {user_id_example}:")
print(top_k_classifier[['title', 'genres', 'relevance_prob']])

# *MLPRegressor
print("\n* MLPRegressor")

y_train_rating = rated_movies['rating']
mlp_regressor = MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42, activation='relu', solver='adam')
mlp_regressor.fit(X_train, y_train_rating)

predicted_ratings = mlp_regressor.predict(X_test)
unrated_movies['predicted_rating'] = predicted_ratings
top_k_regressor = unrated_movies.sort_values('predicted_rating', ascending=False).head(10)

print(f"Top 10 filmes recomendados (por maior nota prevista) para o usuário {user_id_example}:")
print(top_k_regressor[['title', 'genres', 'predicted_rating']])

* MLPClassifier
Top 10 filmes recomendados (por probabilidade de relevância) para o usuário 10:
                                                   title  \
10123           Horror Planet (a.k.a. Inseminoid) (1981)   
22324                                Fata Morgana (1971)   
5751                              Patema Inverted (2013)   
0                                      Enemy Mine (1985)   
1030                           Killer Shrews, The (1959)   
82810                               Class of 1999 (1990)   
7                                  Happening, The (2008)   
993    Last Circus, The (Balada triste de trompeta) (...   
87629  Rahtree: Flower of the Night (Buppha Rahtree) ...   
68138                Tripping the Rift: The Movie (2008)   

                                         genres  relevance_prob  
10123                Action|Drama|Horror|Sci-Fi        1.000000  
22324                  Documentary|Drama|Sci-Fi        0.999995  
5751   Adventure|Animation|Drama|Fantasy|Sci-